In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import cv2
import numpy as np

In [ ]:
# Define data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
}


In [ ]:
# Load datasets
data_dir = '/content/drive/MyDrive/Face Detection'
image_datasets = {x: datasets.ImageFolder(root=data_dir, transform=data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Define model architecture
model = models.resnet18(weights=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(image_datasets['train'].classes))  # Adjust output size to match number of classes


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 69.5MB/s]


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 10
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

train Loss: 1.7984 Acc: 0.2014
val Loss: 1.5786 Acc: 0.4147
train Loss: 1.4735 Acc: 0.4625
val Loss: 1.1448 Acc: 0.6672
train Loss: 1.2196 Acc: 0.5853
val Loss: 0.8913 Acc: 0.7235
train Loss: 1.0185 Acc: 0.6809
val Loss: 0.6683 Acc: 0.7833
train Loss: 0.8805 Acc: 0.7133
val Loss: 0.5198 Acc: 0.8430
train Loss: 0.7312 Acc: 0.7782
val Loss: 0.3916 Acc: 0.8993
train Loss: 0.6475 Acc: 0.7969
val Loss: 0.3010 Acc: 0.9232
train Loss: 0.5629 Acc: 0.8225
val Loss: 0.2674 Acc: 0.9420
train Loss: 0.4961 Acc: 0.8515
val Loss: 0.2307 Acc: 0.9488
train Loss: 0.4609 Acc: 0.8584
val Loss: 0.1725 Acc: 0.9642


In [ ]:
# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/Face Detection/face_recognition_model.pth')

In [ ]:
# Define the model architecture
model = models.resnet18(pretrained=False)  # Assuming you're using ResNet-18
num_classes = 6 # Number of classes (people) in your dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)  # Modify the final fully connected layer

In [ ]:
# Load the trained model state dictionary
model_path = '/content/drive/MyDrive/Face Detection/face_recognition_model.pth'
checkpoint = torch.load(model_path)

In [ ]:
# Load state dictionary onto the model
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
# Set the model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
!pip install Pillow
from PIL import Image

In [ ]:
# Define data transforms for preprocessing the new image
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Function to make predictions on a new image
def predict_image(image_path):
    # Load and preprocess the new image
    image = Image.open(image_path)
    image_tensor = data_transform(image).unsqueeze(0)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        output = model(image_tensor)

    # Post-process the predictions
    probabilities = torch.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)
    predicted_class = predicted_class.item()

    # Map predicted class index to class name or label
    class_names = ['bruce_banner (Mark Ruffalo)', 'clint_barton (Jeremy Renner)', 'Natasha_Romanoff (Scarlett Johansson)', 'steve_rogers (Chris Evans)','thor (Chris Hemsworth)','tony_stark (Robert Downey Jr.)']
    predicted_name = class_names[predicted_class]

    return predicted_name

In [ ]:
image_path = '/content/chris-evans-filming-a-scene-for-captain-america-the-first-avenger-DB5445.jpg'

predicted_person = predict_image(image_path)
print('Predicted Person:', predicted_person)

Predicted Person: tony_stark (Robert Downey Jr.)
